In [1]:
#from pandas import Series, DataFrame
import pandas as pd
import numpy as np
#import os
import matplotlib.pylab as plt
from sklearn.model_selection  import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

In [4]:
prospect_data = pd.read_csv("browsing.csv")
prospect_data.dtypes

SESSION_ID         int64
IMAGES             int64
REVIEWS            int64
FAQ                int64
SPECS              int64
SHIPPING           int64
BOUGHT_TOGETHER    int64
COMPARE_SIMILAR    int64
VIEW_SIMILAR       int64
WARRANTY           int64
SPONSORED_LINKS    int64
BUY                int64
dtype: object

In [5]:
prospect_data.head()

,SESSION_ID,IMAGES,REVIEWS,FAQ,SPECS,SHIPPING,BOUGHT_TOGETHER,COMPARE_SIMILAR,VIEW_SIMILAR,WARRANTY,SPONSORED_LINKS,BUY
0,1001,0,0,1,0,1,0,0,0,1,0,0
1,1002,0,1,1,0,0,0,0,0,0,1,0
2,1003,1,0,1,1,1,0,0,0,1,0,0
3,1004,1,0,0,0,1,1,1,0,0,0,0
4,1005,1,1,1,0,1,0,1,0,0,0,0


In [6]:
#summary statistics 
prospect_data.describe()

,SESSION_ID,IMAGES,REVIEWS,FAQ,SPECS,SHIPPING,BOUGHT_TOGETHER,COMPARE_SIMILAR,VIEW_SIMILAR,WARRANTY,SPONSORED_LINKS,BUY
count,500.000000,500.000000,500.0000,500.000000,500.0000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,1250.500000,0.510000,0.5200,0.440000,0.4800,0.528000,0.500000,0.580000,0.468000,0.532000,0.550000,0.370000
std,144.481833,0.500401,0.5001,0.496884,0.5001,0.499715,0.500501,0.494053,0.499475,0.499475,0.497992,0.483288
min,1001.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1125.750000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1250.500000,1.000000,1.0000,0.000000,0.0000,1.000000,0.500000,1.000000,0.000000,1.000000,1.000000,0.000000
75%,1375.250000,1.000000,1.0000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1500.000000,1.000000,1.0000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
prospect_data.corr()['BUY']

SESSION_ID         0.026677
IMAGES             0.046819
REVIEWS            0.404628
FAQ               -0.095136
SPECS              0.009950
SHIPPING          -0.022239
BOUGHT_TOGETHER   -0.103562
COMPARE_SIMILAR    0.190522
VIEW_SIMILAR      -0.096137
WARRANTY           0.179156
SPONSORED_LINKS    0.110328
BUY                1.000000
Name: BUY, dtype: float64

In [9]:
#Drop columns with low correlation
predictors = prospect_data[['REVIEWS','COMPARE_SIMILAR','WARRANTY','SPONSORED_LINKS']]
target = prospect_data.BUY

In [11]:
#Test and Train split
pred_train, pred_test, tar_train, tar_test  =   train_test_split(predictors, target, test_size=.3)

print( "Predictor - Training : ", pred_train.shape, "Predictor - Testing : ", pred_test.shape )

Predictor - Training :  (350, 4) Predictor - Testing :  (150, 4)


In [12]:
#Gaussian Classifier - Naive Bayes Algo
from sklearn.naive_bayes import GaussianNB

classifier=GaussianNB()
classifier=classifier.fit(pred_train,tar_train)

predictions=classifier.predict(pred_test)

#Analyze accuracy of predictions
sklearn.metrics.confusion_matrix(tar_test,predictions)

array([[79, 15],
       [28, 28]], dtype=int64)

In [ ]:
# calculation of Accuracy with accuracy_score command
# Accuracy = ((79+28)/150) = 71.33%
# accuracy =(((Predicted No, Actual No) value+ (Actual Yes, Predicted Yes) value) divide by testing total (i.e. 150 in our example))
# Error Rate = 1- Accuracy= 28.66 %
# False Positive (Type I error) = 15.95 %
# Precision =(15/43)=34.88 %


In [13]:
# we don't need accuracy score command we can calculate it from above confusion matrix
sklearn.metrics.accuracy_score(tar_test, predictions)

0.71333333333333337

In [14]:
# Probability that prospect will buy
pred_prob=classifier.predict_proba(pred_test)
pred_prob[0,1]

0.36226734087211621

In [ ]:
#above probability conveys that 36 % chance that the customer will buy the product

In [19]:
browsing_data = np.array([0,0,0,0]).reshape(1, -1)
print("New visitor: propensity :",classifier.predict_proba(browsing_data)[:,1] )

New visitor: propensity : [ 0.0319149]


In [21]:
browsing_data = np.array([0,1,0,0]).reshape(1, -1)
print("After checking similar products: propensity :",classifier.predict_proba(browsing_data)[:,1] )

After checking similar products: propensity : [ 0.08908823]


In [23]:
browsing_data = np.array([1,1,0,0]).reshape(1, -1)
print("After checking reviews: propensity :",classifier.predict_proba(browsing_data)[:,1] )

After checking reviews: propensity : [ 0.4960786]


In [24]:
browsing_data = np.array([1,0,1,0]).reshape(1, -1)
print("After checking warranty: propensity :",classifier.predict_proba(browsing_data)[:,1] )

After checking warranty: propensity : [ 0.44452862]


In [ ]:
# Lets assume our propensity threshold to be 50%

In [27]:
browsing_data = np.array([1,1,1,0]).reshape(1, -1)
print("After checking reviews and warranty: propensity :",classifier.predict_proba(browsing_data)[:,1] )

After checking reviews and warranty: propensity : [ 0.70362619]
